# sd-tsia204_lab2_sukeratha_alexandre
Author : SUKERATHA Alexandre

In [79]:
# Change here using YOUR own first and last names
fn1 = "alexandre"
ln1 = "sukeratha"
filename = "_".join(map(lambda s: s.strip().lower(),
["SD-TSIA204_lab2", ln1, fn1])) + ".ipynb"

# Printing filename 
print(f'filename : {filename}')

filename : sd-tsia204_lab2_sukeratha_alexandre.ipynb


## Importing libraries

In [80]:
# Data basics
import math
import numpy as np
import pandas as pd

# ML - Sklearn essentials
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

# ML - Sklearn models
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SequentialFeatureSelector

# Plot 
import plotly.express as px
from tabulate import tabulate
import matplotlib.pyplot as plt

# StatsModel
from scipy import stats
from scipy.stats import t
from scipy.stats import f
import statsmodels.api as sm

# Warnings
import warnings
warnings.filterwarnings("ignore")

***
## Table of contents
* **[Part 1. Preprocess the Data](#chapter1)**
    * [1.a Random seed](#section_1_a)
    * [1.b Load the Data](#section_1_b)
    * [1.c Standardizing the Data](#section_1_c)
    * [1.d Regular OLS regression](#section_1_d)
    * [1.e Store the R2 coefficient in df_coef](#section_1_e)
* **[Part 2. Variable Selection](#chapter2)**
    * [2. Forward Variable Selection method](#section_2)
    * [3.a OLS regression on the variables with a p-value smaller than 0.05](#section_3_a)
    * [3.b Store the R2 coefficient in df_coef](#section_3_b)
    * [4.a Use SequentialFeatureSelector on a linear regression estimator select](#section_3_b)
* **[Part 3. Ridge](#chapter3)**
    * [5. Ridge estimator](#section_5)
    * [5.a Plot how the values of the coefficients change with $\alpha$](#section_5_a)
    * [5.b Plot how MSE of both the train and test sets change with $\alpha$](#section_5_b)
    * [5.c Store the R2 coefficient in df_coef](#section_5_c)
* **[Part 4. Crossvalidation, Lasso and elastic net](#chapter4)**
    * [6.a sklearn version of the Lasso](#section_6_a)
    * [6.b Plot the number of coefficients that are different from $0$ for each value of $\alpha$](#section_6_b)
    * [6.c Plot how MSE of both the train and test sets change with $\alpha$](#section_6_c)
    * [7. CrossValidation](#section_7)
* **[Part 5. Bootstrap](#chapter5)**
    * [8. Regression lines](#section_8)
* **[Part 6. PCA](#chapter6)**
    * [9. Covariance matrix and SVD](#section_9)
    * [9.a heatmap of the covariance matrix](#section_9_a)
    * [9.b PCA Data transformation](#section_9_b)
    * [9.c Plot the amount of variance explained by the first $k$ components](#section_9_c)
    * [9.d First 2 PC approximation](#section_9_d)
    * [9.e Run OLS on the projected Data](#section_9_e)
* **[Part 7. Comparison of the models](#chapter7)**

***
## Part 1. Preprocess the Data <a class="anchor" id="chapter1"></a>

### 1.a Random seed <a class="anchor" id="section_1_a"></a>

In [81]:
np.random.seed(0)

### 1.b Load the Data <a class="anchor" id="section_1_b"></a>

In [82]:
df = pd.read_csv('meatspec.csv')
df.head(3)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V92,V93,V94,V95,V96,V97,V98,V99,V100,fat
0,2.61776,2.61814,2.61859,2.61912,2.61981,2.62071,2.62186,2.62334,2.62511,2.62722,...,2.98145,2.96072,2.94013,2.91978,2.89966,2.87964,2.85960,2.83940,2.81920,22.5
1,2.83454,2.83871,2.84283,2.84705,2.85138,2.85587,2.86060,2.86566,2.87093,2.87661,...,3.29186,3.27921,3.26655,3.25369,3.24045,3.22659,3.21181,3.19600,3.17942,40.1
2,2.58284,2.58458,2.58629,2.58808,2.58996,2.59192,2.59401,2.59627,2.59873,2.60131,...,2.68951,2.67009,2.65112,2.63262,2.61461,2.59718,2.58034,2.56404,2.54816,8.4


We then compute the mean, and standard deviation of every covariate.

In [83]:
headers = ["Variable", "Mean", "Standard Deviation"]
table = []

for var in df.columns.values.tolist()[:-1]:
    mean = df[var].mean()
    std_dev = df[var].std()
    table.append([var, mean, std_dev])

print(tabulate(table, headers))

Variable       Mean    Standard Deviation
----------  -------  --------------------
V1          2.80856              0.410793
V2          2.81114              0.413352
V3          2.81373              0.415906
V4          2.81636              0.418465
V5          2.8191               0.42104
V6          2.82198              0.423635
V7          2.82506              0.426245
V8          2.82838              0.428866
V9          2.83194              0.43151
V10         2.83581              0.434195
V11         2.83996              0.436906
V12         2.84439              0.439653
V13         2.84915              0.442446
V14         2.85429              0.445303
V15         2.85984              0.448239
V16         2.86584              0.451257
V17         2.87229              0.454356
V18         2.87924              0.457546
V19         2.88673              0.460843
V20         2.89469              0.464272
V21         2.90299              0.467797
V22         2.91136              0.4

In [84]:
# Describing the DataSet
df.describe().iloc[:2]

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V92,V93,V94,V95,V96,V97,V98,V99,V100,fat
count,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,215.000000,...,215.000000,215.000000,215.000000,215.000000,215.000000,215.00000,215.00000,215.000000,215.000000,215.000000
mean,2.808561,2.811137,2.813727,2.816363,2.819098,2.821983,2.825064,2.828375,2.831943,2.835813,...,3.178262,3.158254,3.138534,3.119104,3.099971,3.08107,3.06229,3.043548,3.024895,18.142326


The Data is not centered therefore not normalized. Values are not comprised between $0$ and $1$, the Data isn't standardized.

### 1.c Standardizing the Data <a class="anchor" id="section_1_c"></a>

In [85]:
# Dependant and explicative variables
x = df.drop(["fat"], axis=1)
y = df["fat"]

# Splitting the data in train and test sets
x_training_data, x_test_data, y_training_data, y_test_data = train_test_split(
    x, y, test_size=0.25, random_state=0
)

# Standardizing the data
scaler = StandardScaler()

x_training_data_std = scaler.fit_transform(x_training_data)
x_test_data_std = scaler.fit_transform(x_test_data)

y_test_data = stats.zscore(y_test_data)
y_training_data = stats.zscore(y_training_data)

### 1.d Regular OLS regression <a class="anchor" id="section_1_d"></a>

In [86]:
# Fit linear regression model using sklearn
model = LinearRegression(fit_intercept=True).fit(x_training_data_std, y_training_data)

# Printing calculated intercept
headers = ['Variable', 'Value']
table = [['Model intercept', model.intercept_]]
print(tabulate(table, headers))

Variable               Value
---------------  -----------
Model intercept  6.19087e-12


As the calculated value indicates, there is no need to calculate the intercept when we run our OLS model.

### 1.e Store the R2 coefficient in `df_coef` <a class="anchor" id="section_1_e"></a>

In [87]:
# OLS model
model = LinearRegression(fit_intercept=True).fit(x_training_data_std, y_training_data)

# Computing R2 scores
r2_train = r2_score(y_training_data, model.predict(x_training_data_std))
r2_test = r2_score(y_test_data, model.predict(x_test_data_std))

# Storing the scores in a DataFrame
df_coef = pd.DataFrame(columns=['model', 'subset', 'r2 value'])

df_coef = df_coef.append(
    {"r2 value": r2_train, "model": "Baseline OLS regression", "subset":"train",},
    ignore_index=True,
)
df_coef = df_coef.append(
    {"r2 value": r2_test, "model": "Baseline OLS regression", "subset":"test",},
    ignore_index=True,
).drop_duplicates()

# Printing DataFrame
df_coef

,model,subset,r2 value
0,Baseline OLS regression,train,0.995789
1,Baseline OLS regression,test,0.86658


***
## Part 2. Variable selection <a class="anchor" id="chapter2"></a>

### 2. Forward Variable Selection method <a class="anchor" id="section_2"></a>

* **Definition :** `ForwardVariableSelection` is a stepwise regression that starts with an empty model and adds variables one by one. At each step, we add the variable that brings the best improvement to your model. 

* **Implementation :** For our implementation, we will use the student test as the improvement criterion. At each iteration, the feature with the highest absolute value of the student test will be added to the model.

In [88]:
class ForwardVariableSelection:
    def __init__(self, nb_features=-1):
        # Default : all features considered
        self.nb_features = nb_features

    def fit(self, X, y):
        features = np.arange(0, X.shape[1]).tolist()
        arrival_list, p_values = [], []
        # Initiating the number of features asked
        if self.nb_features == -1:
            stop = 0
        else:
            stop = X.shape[1] - self.nb_features
        while len(features) > stop:
            t_max = -math.inf
            for feature in features:
                x_fvs = X[:, arrival_list + [feature]]
                # OLS regression
                model_fvs = sm.OLS(y, sm.add_constant(x_fvs)).fit()
                # Retrieving t-test and p-value
                p_val = model_fvs.pvalues[-1]
                t_val = model_fvs.tvalues[-1]
                # Searching for the max
                if abs(t_val) >= t_max:
                    max_feature = [feature, p_val]
                    t_max = abs(t_val)
            # Adding the feature to our list of arrivals
            arrival_list.append(max_feature[0])
            # Removing the feature from the list of remaining features
            features.remove(max_feature[0])
            # Adding the p-value to our results list
            p_values.append(max_feature)
        self.arrival_list = arrival_list
        self.p_values = p_values
        return self

We apply the Forward Variable Selection method to out standardized DataSet. We store the order of the variable selection and
the associated p-value for each of them in a list `select_order`.

In [89]:
select_order = (
    ForwardVariableSelection().fit(x_training_data_std, y_training_data).p_values
)
df_select_order = pd.DataFrame(select_order, columns=["Variable", "p-value"])

# Print DataFrame
df_select_order.head(5)

,Variable,p-value
0,40,9.966887e-15
1,19,2.149156e-60
2,47,2.431086e-26
3,21,3.235209e-08
4,0,2.474025e-03


### 3.a OLS regression on the variables with a p-value smaller than 0.05 <a class="anchor" id="section_3_a"></a>

We store the significant variables (p-value less than 0.05) in a list `significant_variables`.

In [90]:
significant_variables = []
for pval in select_order:
    if pval[1] < 0.05:
        significant_variables.append(pval[0])

# Length -> Significant variables
headers = ["Variable", "Value"]
table = [["Number of significant variables", len(significant_variables)]]
print(tabulate(table, headers))

Variable                           Value
-------------------------------  -------
Number of significant variables       34


We then run an OLS regression on these variables. 

In [91]:
x_training_fvs, x_test_fvs = (
    x_training_data_std[:, significant_variables],
    x_test_data_std[:, significant_variables],
)
model = LinearRegression(fit_intercept=True).fit(x_training_fvs, y_training_data)

### 3.b Store the R2 coefficient in `df_coef` <a class="anchor" id="section_3_b"></a>

We store the R2 coefficient of our new model in the dataFrame `df_coef`.

In [92]:
# OLS model
x_training_fvs, x_test_fvs = x_training_data_std[:, significant_variables], x_test_data_std[:, significant_variables]
model = LinearRegression(fit_intercept=True).fit(x_training_fvs, y_training_data)

# Computing R2 scores
r2_train = r2_score(y_training_data, model.predict(x_training_fvs))
r2_test = r2_score(y_test_data, model.predict(x_test_fvs))

# Storing the scores in a DataFrame
df_coef = df_coef.append(
    {"r2 value": r2_train, "model": "OLS - Forward Variable Selection", "subset":"train",},
    ignore_index=True,
)
df_coef = df_coef.append(
    {"r2 value": r2_test, "model": "OLS - Forward Variable Selection", "subset":"test",},
    ignore_index=True,
).drop_duplicates()

# Printing DataFrame
df_coef

,model,subset,r2 value
0,Baseline OLS regression,train,0.995789
1,Baseline OLS regression,test,0.86658
2,OLS - Forward Variable Selection,train,0.986968
3,OLS - Forward Variable Selection,test,0.844328


### 4.a Use SequentialFeatureSelector on a linear regression estimator select <a class="anchor" id="section_4_a"></a>

In [93]:
# Sequential Feature Selector
sfs = SequentialFeatureSelector(
    estimator=LinearRegression(fit_intercept=True),
    n_features_to_select=len(significant_variables),
    direction="forward",
)

sfs.fit(x_training_data_std, y_training_data)

SequentialFeatureSelector(estimator=LinearRegression(), n_features_to_select=34)

We retrieve the list of indices using built-in functions. We then store the R2 coefficient of our new model in the dataFrame `df_coef`.

In [94]:
# Conversion list_var to index_list
index_list = sfs.get_support(indices=True)

# OLS regression
x_training_sfs, x_test_sfs = (
    x_training_data_std[:, index_list],
    x_test_data_std[:, index_list],
)
model = LinearRegression(fit_intercept=True).fit(x_training_sfs, y_training_data)

# Computing R2 scores
r2_train = r2_score(y_training_data, model.predict(x_training_sfs))
r2_test = r2_score(y_test_data, model.predict(x_test_sfs))

# Storing the scores in a DataFrame
df_coef = df_coef.append(
    {
        "r2 value": r2_train,
        "model": "OLS - Sequential Feature Selector",
        "subset": "train",
    },
    ignore_index=True,
)
df_coef = df_coef.append(
    {
        "r2 value": r2_test,
        "model": "OLS - Sequential Feature Selector",
        "subset": "test",
    },
    ignore_index=True,
).drop_duplicates()

# Printing DataFrame
df_coef


,model,subset,r2 value
0,Baseline OLS regression,train,0.995789
1,Baseline OLS regression,test,0.86658
2,OLS - Forward Variable Selection,train,0.986968
3,OLS - Forward Variable Selection,test,0.844328
4,OLS - Sequential Feature Selector,train,0.98803
5,OLS - Sequential Feature Selector,test,0.86311


According to sklearn's library on [SequentialFeatureSelector](https://scikit-learn.org/stable/modules/generated/sklearn.feature_selection.SequentialFeatureSelector.html#:~:text=This%20Sequential%20Feature%20Selector%20adds,validation%20score%20of%20an%20estimator.), the sklearn implementation uses the estimator score method as an improvement criterion. In the case of OLS regression, this is r2. In this way, the sklearn implementation differs from our home-made Forward Variable Selector.

***
## Part 3. Ridge <a class="anchor" id="chapter3"></a>

We consider the following minimization problem, 
$$
||Y-X\theta||^2+n\lambda||\theta||^2
$$

### 5. Ridge estimator <a class="anchor" id="section_5"></a>

The minimizer of the previous optimization problem exists and is unique. It is given by, 
$$
\hat{\theta}^{\text{(rdg)}}=(X^TX+n\lambda I_p)^{-1}X^TY
$$
We now implement the estimator in the class `RidgeRegression`. 

In [95]:
class RidgeRegression:
    def __init__(self, alpha):
        self.alpha = alpha

    def fit(self, X, y):
        X_intercept = sm.add_constant(X)
        A = self.alpha * np.identity(X_intercept.shape[1])
        A[0, 0] = 0
        coef_ = (
            np.linalg.inv(X_intercept.T.dot(X_intercept) + A).dot(X_intercept.T).dot(y)
        )
        self.coef_ = coef_
        self.intercept_ = self.coef_[0]
        return self

    def predict(self, X):
        X_predict = sm.add_constant(X)
        self.predictions = X_predict.dot(self.coef_).tolist()
        return self.predictions

    def compute_MSE(self, X, y):
        X_predict = sm.add_constant(X)
        y_check = y.tolist()
        self.predictions = X_predict.dot(self.coef_).tolist()
        self.MSE = 0
        for i in range(0, len(self.predictions)):
            self.MSE += (self.predictions[i] - y_check[i]) ** 2
        return self.MSE

### 5.a Plot how the values of the coefficients change with $\alpha$ <a class="anchor" id="section_5_a"></a>

In [96]:
# alpha scale
A, alphas = [], [10 ** (-i) for i in range(9, 1, -1)]

# For alpha in alphas : compute the values of the coefficients
for alpha in alphas:
    coef = RidgeRegression(alpha).fit(x_training_data_std, y_training_data).coef_
    A.append(coef)
A = np.array(A)

# Store all values in a DataFrame for plotting
df_plot_1 = pd.DataFrame({"coef_value": [], "alpha": [], "variable": []})
df_alphas = pd.DataFrame(alphas, columns=["alpha"])
for i in range(0, A.shape[1]):
    df_coef_values = pd.DataFrame(A[:, i], columns=["coef_value"])
    df_concat = pd.concat((df_coef_values, df_alphas), axis=1)
    df_concat["variable"] = "V" + str(i)
    df_plot_1 = pd.concat((df_plot_1, df_concat), axis=0)

# Displaying values
px.line(
    df_plot_1,
    x="alpha",
    y="coef_value",
    log_x=True,
    color="variable",
    title="figure 1 : <b>Ridge Estimator - evolution of coefficients with alpha</b><br><i>(V0 corresponds to the constant)</i>",
)

### 5.b Plot how MSE of both the train and test sets change with $\alpha$ <a class="anchor" id="section_5_b"></a>

In [97]:
# RidgeRegression model
model_RidgeReg = (
    RidgeRegression(alpha)
    .fit(x_training_data_std, y_training_data)
    .predict(x_test_data)
)

# Computing MSE values
MSE_train_values, MSE_test_values = [], []
for alpha in alphas:
    MSE_test = (
        RidgeRegression(alpha)
        .fit(x_training_data_std, y_training_data)
        .compute_MSE(x_test_data_std, y_test_data)
    )
    MSE_train = (
        RidgeRegression(alpha)
        .fit(x_training_data_std, y_training_data)
        .compute_MSE(x_training_data_std, y_training_data)
    )
    MSE_test_values.append(MSE_test)
    MSE_train_values.append(MSE_train)

# Store all values in a DataFrame for plotting
df_MSE_test_values = pd.DataFrame(MSE_test_values, columns=["MSE_value"])
df_concat_test = pd.concat((df_MSE_test_values, df_alphas), axis=1)
df_concat_test["Data Set"] = "test set"
df_MSE_train_values = pd.DataFrame(MSE_train_values, columns=["MSE_value"])
df_concat_train = pd.concat((df_MSE_train_values, df_alphas), axis=1)
df_concat_train["Data Set"] = "train set"
df_plot_2 = pd.concat((df_concat_test, df_concat_train), axis=0)

# Displaying values
fig_2 = px.line(
    df_plot_2,
    x="alpha",
    y="MSE_value",
    log_x=True,
    markers=True,
    color="Data Set",
    title="figure 2 : <b>Ridge Estimator - MSE as a function of alpha</b><br><i>(We considered both training and testing data when calculating the MSE)</i>",
)

# Signaling the minimum
fig_2.add_vline(x=10 ** (-5), line_width=2, line_dash="dash", line_color="gray")
fig_2.add_hline(y=4.560085, line_width=2, line_dash="dash", line_color="gray")

fig_2.show()

### 5.c Store the R2 coefficient in `df_coef` <a class="anchor" id="section_5_c"></a>

In [98]:
# OLS regression
model = RidgeRegression(alpha=10 ** (-9)).fit(x_training_data_std, y_training_data)

# Computing R2 scores
r2_train = r2_score(y_training_data, model.predict(x_training_data_std))
r2_test = r2_score(y_test_data, model.predict(x_test_data_std))

# Storing the scores in a DataFrame
df_coef = df_coef.append(
    {
        "r2 value": r2_train,
        "model": "Ridge regression",
        "subset": "train",
    },
    ignore_index=True,
)
df_coef = df_coef.append(
    {
        "r2 value": r2_test,
        "model": "Ridge regression",
        "subset": "test",
    },
    ignore_index=True,
).drop_duplicates()

# Printing DataFrame
df_coef


,model,subset,r2 value
0,Baseline OLS regression,train,0.995789
1,Baseline OLS regression,test,0.86658
2,OLS - Forward Variable Selection,train,0.986968
3,OLS - Forward Variable Selection,test,0.844328
4,OLS - Sequential Feature Selector,train,0.98803
5,OLS - Sequential Feature Selector,test,0.86311
6,Ridge regression,train,0.995741
7,Ridge regression,test,0.876367


***
## Part 4. Crossvalidation, Lasso and elastic net <a class="anchor" id="chapter4"></a>

### 6.a sklearn version of the Lasso <a class="anchor" id="section_6_a"></a>

When setting `tol` and `max_iter`, we have to take the following points into account:
* `tol` controls how close you want to be to your optimum. Therefore, reducing the `tol` increases both the accuracy of the solution and the computation time.
* `max_iter` refers to the maximum number of iterations the algorithm will perform to reach a solution. Therefore, increasing `max_iter` increases both the accuracy of the solution and the computation time.

### 6.b Plot the number of coefficients that are different from $0$ for each value of $\alpha$ <a class="anchor" id="section_6_b"></a>

In [99]:
# LassoRegression : number of coefficients different from 0 for each alpha
non_zero_coef, alphas_2 = [], np.logspace(-5, -2, 50)
for alpha in alphas_2:
    LassoRegression = linear_model.Lasso(
        alpha=alpha, fit_intercept=True, max_iter=10**6, tol=1e-1
    )
    LassoRegression.fit(x_training_data_std, y_training_data)
    non_zero_coef.append(np.count_nonzero(LassoRegression.coef_))

# Conversion to DataFrame for plotting
df_alphas_2 = pd.DataFrame(alphas_2, columns=["alpha"])
df_not_null = pd.DataFrame(non_zero_coef, columns=["nb. not null coeff"])
df_plot_3 = pd.concat((df_not_null, df_alphas_2), axis=1)

# Display values
fig_3 = px.line(
    df_plot_3,
    x="alpha",
    y="nb. not null coeff",
    log_x=True,
    markers=True,
    title="figure 3 : <b>Lasso Estimator - Evolution of the number of not null coeff. as a function of <i>alpha</i>",
)
fig_3.show()


### 6.c Plot how MSE of both the train and test sets change with $\alpha$ <a class="anchor" id="section_6_c"></a>

In [100]:
# MSE of both the train and test sets change with alpha
MSE_train_values, MSE_test_values = [], []
for alpha in alphas:
    # Lasso Regression with alpha
    LassoRegression = linear_model.Lasso(
        alpha=alpha, fit_intercept=True, tol=1e-1, max_iter=10**6
    )
    LassoRegression.fit(x_training_data_std, y_training_data)
    # MSE values
    MSE_test_values.append(
        mean_squared_error(y_test_data, LassoRegression.predict(x_test_data_std))
    )
    MSE_train_values.append(
        mean_squared_error(
            y_training_data, LassoRegression.predict(x_training_data_std)
        )
    )

# Converting to DataFrame for plotting
df_MSE_test_values = pd.DataFrame(MSE_test_values, columns=["MSE_value"])
df_concat_test = pd.concat((df_MSE_test_values, df_alphas), axis=1)
df_concat_test["Data Set"] = "test set"

df_MSE_train_values = pd.DataFrame(MSE_train_values, columns=["MSE_value"])
df_concat_train = pd.concat((df_MSE_train_values, df_alphas), axis=1)
df_concat_train["Data Set"] = "train set"

df_plot_4 = pd.concat((df_concat_test, df_concat_train), axis=0)

# Displaying figure
fig_4 = px.line(
    df_plot_4,
    x="alpha",
    y="MSE_value",
    log_x=True,
    markers=True,
    color="Data Set",
    title="figure 4 : <b>Lasso Estimator - MSE as a function of alpha</b><br><i>(We considered both training and testing data for prediction)</i>",
)

fig_4.show()

We then store the R2 coefficient in `df_coef`.

In [101]:
# OLS regression
model = linear_model.Lasso(alpha=0.01, fit_intercept=True, max_iter=10**3).fit(
    x_training_data_std, y_training_data
)

# Computing R2 scores
r2_train = r2_score(y_training_data, model.predict(x_training_data_std))
r2_test = r2_score(y_test_data, model.predict(x_test_data_std))

# Storing the scores in a DataFrame
df_coef = df_coef.append(
    {
        "r2 value": r2_train,
        "model": "Lasso regression",
        "subset": "train",
    },
    ignore_index=True,
)
df_coef = df_coef.append(
    {
        "r2 value": r2_test,
        "model": "Lasso regression",
        "subset": "test",
    },
    ignore_index=True,
).drop_duplicates()

# Printing DataFrame
df_coef

,model,subset,r2 value
0,Baseline OLS regression,train,0.995789
1,Baseline OLS regression,test,0.86658
2,OLS - Forward Variable Selection,train,0.986968
3,OLS - Forward Variable Selection,test,0.844328
4,OLS - Sequential Feature Selector,train,0.98803
5,OLS - Sequential Feature Selector,test,0.86311
6,Ridge regression,train,0.995741
7,Ridge regression,test,0.876367
8,Lasso regression,train,0.882924
9,Lasso regression,test,0.849252


### 7. CrossValidation <a class="anchor" id="section_7"></a>

We first implement `CrossValidation` in a manner similar to that done in the sklearn library. This will help usensure that every observation from the original DataSet appears in both training and test subset.

In [102]:
# Auxiliary function
def join_list(list):
    result = []
    for elt in list:
        result = result + elt
    return result


# Cross Validation
class CrossValidation:
    def __init__(self, estimator, X, y, cv=5):
        self.estimator = estimator
        self.X = X
        self.y = y
        self.cv = cv
        self.nb_rows = self.X.shape[0]
        if self.cv > self.nb_rows:
            raise ValueError(
                "The number of folds can not be higher than the number of data rows"
            )

    def compute_scores(self):
        # Creating cv-folds
        self.create_folds_x()
        self.MSE_values = []
        self.r2_values = []
        # Computing MSE for each test fold
        for i in range(0, len(self.folds_x)):
            x_folds, y_folds = self.folds_x.copy(), self.folds_y.copy()
            x_test, y_test = x_folds[i], y_folds[i]
            x_folds.pop(i), y_folds.pop(i)
            x_train, y_train = join_list(x_folds), join_list(y_folds)
            # Model
            model = self.estimator.fit(x_train, y_train)
            # Storing scores
            self.MSE_values.append(mean_squared_error(y_test, model.predict(x_test)))
            self.r2_values.append(r2_score(y_test, model.predict(x_test)))
        # Calculating the average score
        sum_scores_MSE, sum_scores_r2 = 0, 0
        for i in range(0, len(self.MSE_values)):
            sum_scores_MSE += self.MSE_values[i]
            sum_scores_r2 += self.r2_values[i]
        self.avg_score_MSE = sum_scores_MSE / len(self.MSE_values)
        self.avg_score_r2 = sum_scores_r2 / len(self.r2_values)
        return self

    def create_folds_x(self):
        self.folds_x, self.folds_y = [], []
        rest, i = self.nb_rows % self.cv, 0
        self.folds_size = self.nb_rows // self.cv
        while i <= self.cv - 1:
            self.folds_x.append(
                self.X[[i for i in range(i, i + self.folds_size + 1)], :].tolist()
            )
            self.folds_y.append(self.y[i : i + self.folds_size + 1].tolist())
            i += 1
        if rest != 0:
            remaining_rows_x = self.X[-rest:].tolist()
            remaining_rows_y = self.y[-rest:].tolist()
            j = 0
            for row in remaining_rows_x:
                self.folds_x[j].append(row)
                j += 1
            j = 0
            for row in remaining_rows_y:
                self.folds_y[j].append(row)
                j += 1
        return

We use the sklearn version of the Elastic net. We test it for a penalty parameter $\alpha$-ridge
spaced evenly on a log scale $10e^{-10}$ to $10e^3$ and $\alpha$-lasso in $[0, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99]$. We then plot a 3D scatter plot of the r2 score of the elastic net model as a function of $\alpha$-ridge and $\alpha$-lasso.

In [103]:
# X standardized
X_std = StandardScaler().fit_transform(df.drop(["fat"], axis=1))
y = df["fat"]

# Alphas
alphas_lasso = [0, 0.1, 0.5, 0.7, 0.9, 0.95, 0.99]
alphas_ridge = [10 ** (-i) for i in range(10, -4, -1)]

# CrossValidation
MSE_cross_val = []
for alpha_l in alphas_lasso:
    for alpha_r in alphas_ridge:
        avg_score = (
            CrossValidation(
                ElasticNet(alpha=alpha_r, l1_ratio=alpha_l), X_std, y, cv=10
            )
            .compute_scores()
            .avg_score_MSE
        )
        MSE_cross_val.append([alpha_l, alpha_r, avg_score])

# Converting to DataFrame for plotting
df_5 = pd.DataFrame(MSE_cross_val, columns=["alpha Lasso", "alpha Ridge", "MSE score"])

# Displaying Data -> 3D scatter plot
camera = dict(eye=dict(x=-1, y=-1.85, z=0.2))
fig_5 = px.scatter_3d(
    df_5, x="alpha Lasso", y="alpha Ridge", z="MSE score", color="MSE score", log_y=True
)
fig_5.update_layout(
    height=700,
    title="figure 5 : <b>CrossValidation - Average MSE score as a function of alpha Lasso and alpha Ridge</b><br><i>(Model : ElasticNet; hover over Data points for additional information)</i>",
    scene_camera=camera,
)

fig_5.show()

**Comments :** alpha Lasso appears to have little or no impact on the MSE score. In contrast, Ridge alpha seems to have a considerable impact on the MSE score. For an alpha Ridge strictly below $0.01$, the MSE score is quite low, with an average of $17$. When we increase alpha Ridge above $0.01$, the MSE soars to $160$. In this case, Alpha Ridge penalizes the model too much, by regularization, which makes the model inaccurate.

***
## Part 5. Bootsrap <a class="anchor" id="chapter5"></a>

### 8. Regression lines <a class="anchor" id="section_8"></a>

We first plot the dataset and the regression line fitted with the whole sample.

In [104]:
# DataFrame of interest
df_bootstrap = df[['V40', 'fat']]

# Displaying figure
fig_6 = px.scatter(
    df_bootstrap,
    x="V40",
    y="fat",
    trendline="ols",
    title="figure 6 : <b>Bootstrap - OLS Dependent variable (<i>fat</i>); Explicative variable (<i>V40</i>)</b><br><i>(Hover on the figure for additional informations)</i>",
)
fig_6.show()


We then generate $50$ bootstrap samples, for each of the samples we fit a regression model and plot the $50$ estimated regression lines in the same
plot.

In [105]:
# Generating bootstrap samples
size_bootstrap = math.floor(0.15 * df_bootstrap.shape[0])
arr_plot = []
for i in range(1, 51):
    df_loop = df_bootstrap.sample(n=size_bootstrap, replace=True)
    df_loop["sample"] = "S" + str(i)
    arr_loop = df_loop.to_numpy().tolist()
    arr_plot = arr_plot + arr_loop
arr_plot
df_7 = pd.DataFrame(arr_plot, columns=["V40", "fat", "sample"])

# Displaying figure
fig_7 = px.scatter(
    df_7,
    x="V40",
    y="fat",
    trendline="ols",
    color="sample",
    title="figure 7 : <b>Bootstrap - OLS Dependent variable (<i>fat</i>); Explicative variable (<i>V40</i>)</b><br><i>(Sample size : 15% of the original DataSet)</i>",
)
fig_7.show()

***
## Part 6. PCA <a class="anchor" id="chapter6"></a>

### 9. Covariance matrix and SVD <a class="anchor" id="section_9"></a>

We consider the Singular Value Decomposition of the covariance matrix, we denote:
$$
\text{cov}(X) = U.s.V^T
$$

In [106]:
# Covariance matrix
cov_matrix = np.cov(x_training_data_std, rowvar=False)

# Singular Value Decomposition SVD
U, s, V = np.linalg.svd(cov_matrix, full_matrices=False)

### 9.a heatmap of the covariance matrix <a class="anchor" id="section_9_a"></a>

In [107]:
# Displaying figure
fig_8 = px.imshow(cov_matrix)
fig_8.update_layout(title='figure 8 : <b>Covariance matrix heatmap</b><br>(<i>used on the standardized Data - train subset</i>)')
fig_8.show()

### 9.b PCA Data transformation <a class="anchor" id="section_9_b"></a>

The principal components are obtained using the vector $U$ containing the eigenvectors of the covariant matrix. Consider the formula:
$$ X_{\text{p.c.}} = X.U$$

In [108]:
# Principal components
x_train_pc = x_training_data_std@U
x_test_pc = x_test_data_std@U

### 9.c Plot the amount of variance explained by the first $k$ components <a class="anchor" id="section_9_c"></a>

In [109]:
# Calculating the amount of variance explained by the first k components
total_var, var_cumulative = np.sum(s), 0
var_explained, k = [], 0
for var in s:
    var_cumulative += var
    var_explained.append(["0-" + str(k), var_cumulative / total_var])
    k += 1

# Conversion to DataFrame for plotting
df_9 = pd.DataFrame(var_explained, columns=["components", "var explained"])

# Displaying figure
fig_9 = px.bar(
    df_9,
    x="components",
    y="var explained",
    title="<b>figure 9: Cumulative variance explained by the first k components as a percentage</b><br><i>(Note that the first component already explains more than 98% of the variance)</i>",
)
fig_9.show()

### 9.d First 2 PC approximation <a class="anchor" id="section_9_d"></a>

In [110]:
# Extracting the two first PC
df_PCA = pd.DataFrame(x_train_pc)
df_PCA_sub = df_PCA[[0, 1]]
df_PCA_sub = pd.concat((df_PCA_sub, df["fat"]), axis=1)

# Displaying figure
fig_10 = px.scatter(
    df_PCA_sub,
    x=0,
    y=1,
    labels={"0": "PCA component 0", "1": "PCA component 1"},
    title="figure 10: <b>Scatter plot, two first PCA components - training subset</b><br><i>(Components were computed using the SVD decomposition on the covariance matrix)</i>",
    color="fat",
    height=700,
)
fig_10.show()

**Comments :** Unfortunately, the 2D graph of the first two components does not allow to find groups or patterns regarding the fat index of the meat pieces.

### 9.e Run OLS on the projected Data <a class="anchor" id="section_9_e"></a>

In [111]:
# Computing r2 scores
r2_pca_test = []
for i in range(1, x_train_pc.shape[1]):
    model = LinearRegression(fit_intercept=True).fit(x_train_pc[:, :i], y_training_data)
    r2 = r2_score(y_test_data, model.predict(x_test_pc[:, :i]))
    r2_pca_test.append(["0-" + str(i), r2, "test"])

# Converting to DataFrame for plotting
df_11 = pd.DataFrame(r2_pca_test, columns=["components", "r2", "subset"])

# Displaying r2 scores
fig_11 = px.line(
    df_11,
    x="components",
    y="r2",
    markers=True,
    title="figure 11: <b>LinearRegression using PCA - r2 score as a function of the number of p.c. taken into account </b><br><i>(Components were computed using the SVD decomposition on the covariance matrix - train subset)</i>",
)
fig_11.show()

## Removing beginning outliers for a better visualization of the maximum
df_12 = df_11.tail(df_11.shape[0] - 3)
fig_12 = px.line(
    df_12,
    x="components",
    y="r2",
    markers=True,
    title="figure 12 : <b>LinearRegression using PCA - r2 score as a function of the number of p.c. taken into account </b><br><i>(We removed outliers r2 for better visualization : k<5)</i>",
)
# Signaling the maximum
fig_12.add_vline(x="0-38", line_width=2, line_dash="dash", line_color="gray")
fig_12.add_hline(
    y=r2_pca_test[37][1], line_width=2, line_dash="dash", line_color="gray"
)

fig_12.show()

We then store the best result in the DataFrame `df_coef`.

In [112]:
# Storing the scores in a DataFrame
df_coef = df_coef.append(
    {
        "r2 value": r2_pca_test[37][1],
        "model": "LinearRegression - PCA",
        "subset": "test",
    },
    ignore_index=True,
).drop_duplicates()

# Printing DataFrame
df_coef

,model,subset,r2 value
0,Baseline OLS regression,train,0.995789
1,Baseline OLS regression,test,0.86658
2,OLS - Forward Variable Selection,train,0.986968
3,OLS - Forward Variable Selection,test,0.844328
4,OLS - Sequential Feature Selector,train,0.98803
5,OLS - Sequential Feature Selector,test,0.86311
6,Ridge regression,train,0.995741
7,Ridge regression,test,0.876367
8,Lasso regression,train,0.882924
9,Lasso regression,test,0.849252


***
## Part 7. Comparison of the models <a class="anchor" id="chapter7"></a>

### **Model comparison table - Benefits and Drawbacks**

|Model|R2 Test Subset|R2 Train Subset|Benefits|Drawbacks|
|---    |:-:    |:-:   |---   |---   |
|Baseline OLS regression|<span style="color:red">0.86658</span>|<span style="color:green">0.995789</span>|- Best Unbiased Linear Estimator<br>- Best results on train subset|- Sensitive to outliers<br>- Prone to underfitting<br>- Assumes that the Data is independant|
|OLS - Forward Variable Selection|0.986968|<span style="color:red">0.844328</span>|- Eliminates irrelevant variables<br>- Reduces computation time|- Worst results on train subset<br>- Information loss|
|OLS - Sequential Feature Selector|0.98803|0.86311|- Eliminates irrelevant variables<br>- Reduces computation time|- Information loss|
|Ridge regression|<span style="color:green">0.995741</span>|0.876367|- Tends to avoid over-fitting<br>- Best results on test subset|- Tends to give small but well distributed weights<br>- Increasing overfitting risk when the number of observations is insufficient|
|Lasso regression|0.882924|0.849252|- Tends to avoid over-fitting|- Tends to give sparse weights<br>- Increasing overfitting risk when the number of observations is insufficient|
|PCA|91.68598|<i>Not calculated</i>|- Removes Correlated Features<br>- Reduces overfitting<br>- 2D visualization|- Information Loss<br>- Variables less interpretable|

<span style="color:green">Green</span> : Best r2 score for [train/test] subset ; <span style="color:red">Red</span> : Worst r2 score for [train/test] subset

```








                          _______
                         | ___  o|
                         |[_-_]_ |
      ______________     |[_____]|
     |.------------.|    |[_____]|
     || >THE END   ||    |[====o]|
     ||            ||    |[_.--_]|
     || >THX 4.    ||    |[_____]|
     || READING :) ||    |      :|
     ||____________||    |      :|
 .==.|""  ......    |.==.|      :|
 |::| '-.________.-' |::||      :|
 |''|  (__________)-.|''||______:|
 `""`_.............._\""`______
    /:::::::::::'':::\`;'-.-.  `\
   /::=========.:.-::"\ \ \--\   \
   \`""""""""""""""""`/  \ \__)   \
    `""""""""""""""""`    '========'

```